## Loading saved model and training on bioavailability

In [ ]:
#load bioavailabity data
bio_avail_data = pd.read_csv("../Data/Bioavailibility.csv")
#print(bio_avail_data.columns)

In [ ]:
#prepare data like we did for solubility:

# Prepare the oral bioavailability dataset
oral_data = oral_data.dropna(subset=['Oral Bioavailability'])
y_oral = oral_data["Oral Bioavailability"]
smiles_oral = oral_data["SMILES"]
X_oral = generate_features(smiles_oral)

X_train_oral, X_test_oral, y_train_oral, y_test_oral = train_test_split(X_oral, y_oral, test_size=0.2, random_state=42)
y_train_oral = y_train_oral.values.astype(np.float32)
y_test_oral = y_test_oral.values.astype(np.float32)

scaler_oral = StandardScaler()
X_train_oral = scaler_oral.fit_transform(X_train_oral)
X_test_oral = scaler_oral.transform(X_test_oral)

X_train_oral_tensor = torch.tensor(X_train_oral, dtype=torch.float32)
y_train_oral_tensor = torch.tensor(y_train_oral, dtype=torch.float32)
X_test_oral_tensor = torch.tensor(X_test_oral, dtype=torch.float32)
y_test_oral_tensor = torch.tensor(y_test_oral, dtype=torch.float32)

y_train_oral_tensor = y_train_oral_tensor.view(-1, 1)
y_test_oral_tensor = y_test_oral_tensor.view(-1, 1)


In [ ]:
#loading saved model
model = SolubilityPredictor(input_dim, hidden_dim, output_dim)
model.load_state_dict(torch.load('solubility_model.pth'))

# Optionally, modify the final layer if the output dimension is different


In [ ]:
# Fine-tune the model on the new dataset. You might want to use a lower learning rate for fine-tuning.

learning_rate_finetune = 0.0001  # Lower learning rate for fine-tuning
optimizer = optim.Adam(model.parameters(), lr=learning_rate_finetune)

# Create DataLoader for the new dataset
train_dataset_oral = TensorDataset(X_train_oral_tensor, y_train_oral_tensor)
train_loader_oral = DataLoader(train_dataset_oral, batch_size=batch_size, shuffle=True)

# Fine-tune the model...
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader_oral:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

# Evaluate on the new test set
model.eval()
with torch.no_grad():
    predictions = model(X_test_oral_tensor)
    loss = criterion(predictions, y_test_oral_tensor)
    print(f'Test Loss: {loss.item()}')


#### Notes:

Adjust the learning rate: When fine-tuning, it is often beneficial to use a lower learning rate to avoid large updates to the pre-trained weights.

Regularization: You can also consider adding dropout layers or other regularization techniques to avoid overfitting, especially when fine-tuning on a smaller dataset.

Layer freezing: Depending on your problem, you might want to freeze some layers (e.g., the first few layers) and only fine-tune the later layers or the final layer.